In [1]:
import csv
import json
from urllib.parse import urlparse

import requests
import pandas as pd

In [2]:
streaming_raw = './streaming_24M.txt'

# Create the tweets CSV

# CSV headeres
header_tweets = ("create_at,source,tweet_id,user_id,source_status_id,"
                 "source_user_id,lang,hashtags,user_metions_names,"
                 "user_metions_id,media_url,external_url,text\n")
header_user = ("user_id,user_name,screen_name,create_at,verified,"
               "following_count,followers_count,tweets_count,"
               "favourite_count,lang,time_zone,location,profile_image_url,"
               "description\n")

# CSV line templates
line_tweets = '''{1},{2},{0},{3},{4},{5},{6},"{7}","{8}","{9}","{10}","{11}","{12}"\n'''
line_user = '''{0},"{1}",{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},"{13}"\n'''

# Open files and write headers
csv_tweets = open('tweets.csv', 'a')
csv_tweets.write(header_tweets)
csv_user = open('user.csv', 'a')
csv_user.write(header_user)

with open(streaming_raw, 'r') as streaming_file:
    for cnt, raw_data in enumerate(streaming_file):
        print('\rProcessing line: {0}'.format(cnt), end='')
        
        data = json.loads(raw_data)
        
        if 'user' not in data.keys():
            continue
        
        """
        Para tweets.csv
        """
        # Tweet text
        if 'retweeted_status' in data.keys():
            text = data['retweeted_status']['text'].replace('"', "'")
        else:
            text = data['text'].replace('"', "'")
        
        # Hashtag utilizados
        hashtags = []
        for hashtag in data['entities']['hashtags']:
            hashtags.append(hashtag['text'].replace('"', "'"))
        
        # Url de la imagen en el retweet
        if 'retweeted_status' in data.keys():
            if 'media' in data['retweeted_status']['entities'].keys():
                media_url = data['retweeted_status']['entities']['media'][0]['media_url_https']
            elif 'extended_tweet' in data['retweeted_status'].keys():
                if 'media' in data['retweeted_status']['extended_tweet']['entities']:
                    media_url = data['retweeted_status']['extended_tweet']['entities']['media'][0]['media_url_https']
                else:
                    media_url = ''
            else:
                media_url = ''
        # Url de la imagen en el tweet
        elif 'extended_entities' in data.keys():
            media_url = data['extended_entities']['media'][0]['media_url_https']
        else:
            media_url = ''
            
        # Url link externo
        if 'retweeted_status' in data.keys():
            if data['retweeted_status']['entities']['urls']:
                external_url = data['retweeted_status']['entities']['urls'][0]['expanded_url']
            else:
                external_url = ''
        elif data['entities']['urls']:
            external_url = data['entities']['urls'][0]['expanded_url']
        else:
            external_url = ''

        # User mentions
        mentions_name = []
        mentions_id = []
        if 'retweeted_status' in data.keys():
            for user in data['retweeted_status']['entities']['user_mentions']:
                mentions_name.append(user['screen_name'])
                mentions_id.append(user['id'])
        else:
            for user in data['entities']['user_mentions']:
                mentions_name.append(user['screen_name'])
                mentions_id.append(user['id'])
                
        # 
        if 'media' in data['entities'].keys():
            if 'source_status_id' in data['entities']['media'][0].keys():
                source_status_id = data['entities']['media'][0]['source_status_id']
                source_user_id = data['entities']['media'][0]['source_user_id']
        else:
            source_status_id = data['id']
            source_user_id = data['user']['id']


        # Source
        source = data['source'].split('">')[1].split('</')[0]

        csv_tweets.write(line_tweets.format(data['id'], data['created_at'], source, 
                                            data['user']['id'], source_status_id, 
                                            source_user_id, data['lang'],hashtags, 
                                            mentions_name, mentions_id, media_url, 
                                            external_url, text))
        
        """
        Para user.csv
        """
        # Reemplazo las comillas dobles por simples
        user_name = data['user']['name'].replace('"', "'")
        # Algunos campos no tiene info, los reemplazo con ''
        if data['user']['description']:
            description = data['user']['description'].replace('"', "'")
        else:
            description = ''
            
        if data['user']['time_zone']:
            time_zone = data['user']['time_zone'].replace(',', ' ')
        else:
            time_zone = ''
            
        if data['user']['location']:
            location = data['user']['location'].replace(',', ' ')
        else:
            location = ''
            
        # User profile url
        profile_url = data['user']['profile_image_url_https']
        profile_url = profile_url.replace('_normal.', '_400x400.')

        csv_user.write(line_user.format(data['user']['id'], user_name, data['user']['screen_name'], 
                                        data['user']['created_at'], data['user']['verified'], 
                                        data['user']['followers_count'], data['user']['friends_count'], 
                                        data['user']['statuses_count'], data['user']['favourites_count'], 
                                        data['user']['lang'], time_zone, location, profile_url, description))

csv_tweets.close()
csv_user.close()

Processing line: 174956

In [24]:
# Remove duplicate user
df_user = pd.read_csv('user.csv')
df_user = df_user.drop_duplicates()

In [25]:
# Buscando el genero y la edad del usuario

# Mantengo la session
headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': '051ed10f2cda4ecb9ee44cd6aaadc6d8',
}
session = requests.session()
session.headers = headers

params = {'returnFaceAttributes': 'age,gender'}
azure_endpoint = 'https://westus.api.cognitive.microsoft.com/face/v1.0/detect'

def faces_atributs(row):
    body = {'url': row['profile_image_url']}
    response = session.post(azure_endpoint, params=params ,json=body)
    
    faces = json.loads(response.text)
    
    if 'error' in faces:
        return ['', '']
    elif len(faces) == 1:
        return [faces[0]['faceAttributes']['gender'], faces[0]['faceAttributes']['age']]
    else:
        return ['', '']

atributes = df_user[:10].apply(faces_atributs, axis=1)
atributes

0              [, ]
1              [, ]
2              [, ]
3      [male, 79.9]
4              [, ]
5              [, ]
6    [female, 17.9]
7              [, ]
8              [, ]
9    [female, 30.5]
dtype: object

In [21]:
atributes[0]

KeyError: 0

In [17]:
atributes.tolist()

[['', ''],
 ['', ''],
 ['', ''],
 ['male', 79.9],
 ['', ''],
 ['', ''],
 ['female', 17.9],
 ['', ''],
 ['', ''],
 ['female', 30.5]]

In [7]:

#df_user.to_csv('user.csv', index=False)

In [12]:
df_user.to_csv('spam.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [13]:
df_2 = pd.read_csv('spam.csv')
df_2

/home/fnbellomo/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_id,user_name,screen_name,create_at,verified,following_count,followers_count,tweets_count,favourite_count,lang,time_zone,location,profile_image_url,description
0,200690392,Franco,framonteg,Sun Oct 10 00:05:35 +0000 2010,False,72.0,66.0,711.0,9.0,es,Buenos Aires,Buenos Aires,https://pbs.twimg.com/profile_images/842532418...,NaN
1,178082661,Lorena,laloredesluis,Fri Aug 13 20:45:22 +0000 2010,False,2020.0,2221.0,70771.0,9701.0,es,Santiago,Merlo San Luis,https://pbs.twimg.com/profile_images/306471578...,"Escritora, Periodista Amo los animales, Ultra ..."
2,1447656871,tedescolaura,tedescolaura17,Wed May 22 01:02:26 +0000 2013,False,42.0,104.0,916.0,47.0,es,NaN,NaN,https://pbs.twimg.com/profile_images/779673958...,NaN
3,296941365,Claudio Bisutti,ClaudioBisutti,Wed May 11 17:10:15 +0000 2011,False,308.0,497.0,4779.0,14.0,es,Buenos Aires,Rosario,https://pbs.twimg.com/profile_images/844265064...,La lucha no se abandona se despliega con otras...
4,2423066049,Mari!,marishma78,Wed Mar 19 17:57:41 +0000 2014,False,5214.0,342.0,137223.0,186171.0,es,Pacific Time (US & Canada),Frente al campo,https://pbs.twimg.com/profile_images/835399833...,Ignorante
5,256655799,Carolina 📍,OutOfZtyle,Wed Feb 23 20:05:23 +0000 2011,False,581.0,362.0,43723.0,4280.0,en,Buenos Aires,Entre Ríos Argentina,https://pbs.twimg.com/profile_images/841449366...,just like the ocean you change what I see
6,883472144,mariucha,mariuchamariuch,Tue Oct 16 01:50:56 +0000 2012,False,202.0,296.0,8163.0,81.0,es,Brasilia,NaN,https://pbs.twimg.com/profile_images/324382759...,Yo deseo para mi pais libertad seguridad justi...
7,797503862,hilda cuello,hildacuello1,Sun Sep 02 04:33:38 +0000 2012,False,132.0,454.0,4204.0,6160.0,es,NaN,NaN,https://pbs.twimg.com/profile_images/844644018...,NaN
8,1488393252,Candela,ahogadaderazon_,Thu Jun 06 18:35:02 +0000 2013,False,483.0,767.0,5488.0,2302.0,es,Brasilia,Buenos Aires,https://pbs.twimg.com/profile_images/840347367...,"No estaba dormida, estaba EBRIA. La Caverna y ..."
9,1705190376,Celeste Mac Dougall,celestemacdou,Tue Aug 27 17:40:23 +0000 2013,False,636.0,886.0,670.0,567.0,es,NaN,NaN,https://pbs.twimg.com/profile_images/754702144...,Docente. Feminista. Luchando por el #AbortoLeg...


In [4]:
streaming_raw = './streaming_24M.txt'
cnt_lines = 0

with open(streaming_raw, 'r') as streaming_file:
    for cnt, raw_data in enumerate(streaming_file):
        cnt_lines += 1
        
cnt_lines

174957

In [3]:
df_user = pd.read_csv('user.csv')
df_user.head()

,user_id,user_name,screen_name,create_at,verified,following_count,followers_count,tweets_count,favourite_count,lang,time_zone,location,description
0,200690392,Franco,framonteg,Sun Oct 10 00:05:35 +0000 2010,False,72.0,66.0,711.0,9.0,es,Buenos Aires,Buenos Aires,NaN
1,178082661,Lorena,laloredesluis,Fri Aug 13 20:45:22 +0000 2010,False,2020.0,2221.0,70771.0,9701.0,es,Santiago,Merlo San Luis,"Escritora, Periodista Amo los animales, Ultra ..."
2,1447656871,tedescolaura,tedescolaura17,Wed May 22 01:02:26 +0000 2013,False,42.0,104.0,916.0,47.0,es,NaN,NaN,NaN
3,296941365,Claudio Bisutti,ClaudioBisutti,Wed May 11 17:10:15 +0000 2011,False,308.0,497.0,4779.0,14.0,es,Buenos Aires,Rosario,La lucha no se abandona se despliega con otras...
4,2423066049,Mari!,marishma78,Wed Mar 19 17:57:41 +0000 2014,False,5214.0,342.0,137223.0,186171.0,es,Pacific Time (US & Canada),Frente al campo,Ignorante


In [8]:
df_user.shape

(174933, 13)

In [4]:
df = pd.read_csv('tweets.csv')
df.head()

,create_at,source,tweet_id,user_id,source_status_id,source_user_id,lang,hashtags,user_metions_names,user_metions_id,media_url,external_url,text
0,Fri Mar 24 13:54:27 +0000 2017,Twitter Web Client,845272602784485377,200690392,845272300006129664,138814032,und,"['memoriaVerdadYJusticia', 'nuncamas']",[],[],https://pbs.twimg.com/media/C7sCp5hWsAAo08N.jpg,NaN,#memoriaVerdadYJusticia #nuncamas https://t.co...
1,Fri Mar 24 13:54:27 +0000 2017,Twitter Web Client,845272603346571264,178082661,845272603346571264,178082661,es,"['NuncaMas', 'Son30MIL', 'DiaDeLaMemoria', '24...",[],[],https://pbs.twimg.com/media/C7sC3DHXgAEcZiR.jpg,NaN,#NuncaMas\n#Son30MIL\n#DiaDeLaMemoria\n#24DeMa...
2,Fri Mar 24 13:54:27 +0000 2017,Twitter for Windows,845272603455619072,1447656871,845268029785198594,132896453,es,['NuncaMas'],[],[],https://pbs.twimg.com/media/C7r-xp_XgAIZd7s.jpg,NaN,Los gatos no tienen memoria.\n#NuncaMas https:...
3,Fri Mar 24 13:54:28 +0000 2017,Twitter for Android,845272608010592256,296941365,845262374047240193,717624010,es,['NuncaMas'],[],[],https://pbs.twimg.com/media/C7r5o2fX0AA1QCa.jpg,NaN,"#NuncaMas Memoria, Verdad y Justicia. https://..."
4,Fri Mar 24 13:54:28 +0000 2017,Twitter for Android,845272608111296512,2423066049,845272300006129664,138814032,und,"['memoriaVerdadYJusticia', 'nuncamas']",[],[],https://pbs.twimg.com/media/C7sCp5hWsAAo08N.jpg,NaN,#memoriaVerdadYJusticia #nuncamas https://t.co...


In [5]:
df[df['user_metions_names'] != '[]']

,create_at,source,tweet_id,user_id,source_status_id,source_user_id,lang,hashtags,user_metions_names,user_metions_id,media_url,external_url,text
30,Fri Mar 24 13:54:34 +0000 2017,Twitter Web Client,845272631427325953,210210001,845272631427325953,210210001,en,['travesty'],"['SpeakerRyan', 18916432, 'realDonaldTrump', 2...",[],NaN,NaN,@SpeakerRyan @realDonaldTrump @GOP Pray Trumpc...
45,Fri Mar 24 13:54:36 +0000 2017,Twitter for iPhone,845272642282229761,100626120,845272642282229761,100626120,es,['NuncaMas'],"['CFKArgentina', 138814032]",[],NaN,NaN,#NuncaMas Montoneros asesinos robando a 4 mano...
81,Fri Mar 24 13:54:44 +0000 2017,Twitter Web Client,845272674507145216,155034371,845272674507145216,155034371,und,['mACRINONGRATA'],"['nrc', 3103641, 'parool', 32397387, 'eenvanda...",[],NaN,https://twitter.com/i/web/status/8452726745071...,#mACRINONGRATA @nrc @parool @eenvandaag @dwdd ...
86,Fri Mar 24 13:54:45 +0000 2017,Twitter for Android,845272677896081413,730567711017848832,845272677896081413,730567711017848832,es,[],"['CronicaTV', 3816919516]",[],NaN,https://twitter.com/facu_pedrini/status/845268...,Te amo @CronicaTV https://t.co/dm2YwlElRP
92,Fri Mar 24 13:54:46 +0000 2017,Twitter for Android,845272682761457665,2962017483,845272682761457665,2962017483,en,[],"['realDonaldTrump', 25073877]",[],NaN,NaN,"@realDonaldTrump What's so great? Ppl die, u a..."
117,Fri Mar 24 13:54:53 +0000 2017,Twitter for Android,845272709890293761,2189403604,845268637661433856,785919310615064577,es,"['ItaiHagman', '24M', 'NuncaMas', 'MemoriaVerd...","['ItaiHagman', 162465278]",[],https://pbs.twimg.com/ext_tw_video_thumb/84526...,NaN,¿Qué te mueve el 24 de marzo? #ItaiHagman #24M...
123,Fri Mar 24 13:54:53 +0000 2017,Twitter for Android,845272713447034882,205103664,845272713447034882,205103664,es,"['Son30mil', 'NuncaMas']","['DDGALLIANO', 109714931]",[],NaN,NaN,@DDGALLIANO 6348 es lo q cobra un jubilado. Ud...
129,Fri Mar 24 13:54:55 +0000 2017,Twitter for iPhone,845272721370071040,826778480918556673,845272721370071040,826778480918556673,en,[],"['realDonaldTrump', 25073877]",[],NaN,https://twitter.com/i/web/status/8452614532465...,.@realDonaldTrump this bill will:\n1. Raise pr...
133,Fri Mar 24 13:54:55 +0000 2017,Twitter Web Client,845272721869168640,3114991186,845272300006129664,138814032,und,"['memoriaVerdadYJusticia', 'Son30MIL', 'nuncam...","['CFKArgentina', 138814032]",[],https://pbs.twimg.com/media/C7sDCcgXUAAD1hE.jpg,NaN,@CFKArgentina #memoriaVerdadYJusticia #Son30MI...
138,Fri Mar 24 13:54:57 +0000 2017,Twitter Web Client,845272730136203264,4794105166,845272730136203264,4794105166,und,[],"['elisacarrio', 1578961946, 'ceferinoreato', 1...",[],NaN,https://twitter.com/CondenaNacional/status/845...,@elisacarrio @ceferinoreato @mariuvidal https:...


In [42]:
df['lang'].unique()

array(['und', 'es', 'pt', 'en', 'tr', 'it', 'ar', 'hu', 'tl', 'ja', 'fr',
       'in', 'de', 'lt', 'vi', 'fi', 'eu', 'ht', 'nl', 'sl', 'et', 'ru',
       'sv', 'ro', 'ko', 'pl', 'cy', 'no', 'hi', 'da', 'lv', 'is'], dtype=object)

In [52]:
df['source'].unique()

array(['Twitter Web Client', 'Twitter for Windows', 'Twitter for Android',
       'Twitter for iPhone', 'Mobile Web (M5)', 'TweetDeck',
       'Verify your age', ' stereotype', 'PinkHose',
       'Twitter for Windows Phone', 'Linkis: turn sharing into growth',
       'IFTTT', 'Instagram', 'Facebook', 'SuperExamplor',
       'Twitter for iPad', 'Botize', 'TweetCaster for Android', 'Spotify',
       'Mobile Web (M2)', 'Twitter for Mac', 'PoGo_Notify', 'TrendsArab',
       '@pogoredlands', 'Carbon v2', 'الفيْديو للكبآر فقط',
       'DiscordPokemon', 'RoundTeam', 'Twendency-Paperboy',
       'Twitter for Nokia S40', 'Twitter for BlackBerry®', 'Echofon',
       'PogoDTToledoMaumeeRares', 'Plume\xa0for\xa0Android',
       'ToS compliant Twopcharts', 'Es Tendencia en Argentina',
       'App Unfollowers', 'Trends5 Top5 Trends - World',
       'THEELECTROSAURIAN', 'Media Studio', 'RTpapa', 'Hootsuite',
       'TopTrendWWW', 'Fenix for Android', 'Trumperon', 'SylvaniaGen2',
       'Waze Incident

In [19]:
df[df['source'] == 'الفيْديو للكبآر فقط'].iloc[0]['text']

'WHAT THE HECK is she putting inside ?(650)\n#NuncaMas  \nhttps://t.co/A21937URgT'

In [43]:
df.count()

create_at             174840
source                174840
tweet_id              174840
user_id               174840
source_status_id      174840
source_user_id        174840
lang                  174840
hashtags              174840
user_metions_names    174840
user_metions_id       174840
media_url             104685
external_url           69275
text                  174840
dtype: int64

In [51]:
df[df['source_status_id'] == 845300508898316290].iloc[0]['text']

'Memoria Verdad y Justicia #nuncamas #DiaDeLaMemoria https://t.co/tKBjgnjmSi'

In [45]:
c[845267176810598400]

2413

In [44]:
c = df.groupby('source_status_id')['source_status_id'].count()
c.sort_values(ascending=False)

source_status_id
845272300006129664    3234
845267176810598400    2413
845300508898316290    2307
845273239698001920    1892
845268029785198594    1421
845112432527839232    1394
845269676741578753    1034
845260450531041283     787
845234464217321472     714
845374182506582016     674
845272210680041472     604
845320501253885952     590
845267189426982913     521
845266744935616512     455
845246430226866176     423
845263814769696768     416
845126684579790849     406
845311884014309376     341
845395045897375745     336
845297221713170432     328
845318563376369664     284
845401777218469888     283
845264904630276096     283
845308048348450816     268
845307937300078594     254
845217958574129154     242
845364521267089409     237
845416998452576256     218
845272830828822533     216
845267711185899521     212
                      ... 
845397392723726338       1
845397363137155074       1
845397361916612610       1
845397308594421760       1
845397332615139328       1
84539730878

In [ ]:
def unsort_url(urls):
    headers = {
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'User-Agent': ('Mozilla/5.0 (X11; Linux x86_64; rv:45.0)'
                       ' Gecko/20100101 Firefox/45.0'),
    }

    # so connections are recycled
    session = requests.session()
    session.headers = headers

    unsort = {}
    error = 0
    
    for cnt, url in enumerate(urls):
        print("\rFull url {0}/{1}. Errors: {2}".format(cnt, len(urls), error), end='')

        try:
            resp = session.head(url, allow_redirects=True)
            unsort[url] = resp.url
        except:
            error += 1
        
    return unsort


unsort = unsort_url(list(df_sort_url['external_url'].unique()))

In [ ]:
# Contando la cantidad de sort url
sorts_uri = ['http://j.mp/', 'http://bit.ly/', 'http://fb.me/', 'http://ln.is/', 
             'http://goo.gl/', 'http://amzn.to/', 'http://ow.ly/', 'http://tinyurl.com/']
sorts_uri += [uri.replace('http', 'https') for uri in sorts_uri]

def filter_sort_url(ele):
    if ele:
        ele = str(ele)
        parsed_uri = urlparse(ele)
        domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
        
        if domain in sorts_uri:
            return True
        else:
            return False
    else:
        return False

df_tweets = pd.read_csv('./tweets.csv')
df_sort_url = df_tweets[df_tweets['external_url'].apply(filter_sort_url)]
amount_sort_url = len(list(df_sort_url['external_url'].unique()))
amount_sort_url

In [7]:
from collections import Counter

def url_domain(ele):
    if ele:
        parsed_uri = urlparse(ele)
        domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
        return domain
    else:
        return False

df['external_url'] = df['external_url'].fillna(False)
domains = df['external_url'].apply(url_domain)

count = Counter(domains)
df_count = pd.DataFrame([[key, val] for key, val in zip(count.keys(), count.values())], columns=['domain', 'count'])
df_count.sort_values(by='count', ascending=False)

In [46]:
sorts_uri = ['http://j.mp/', 'http://bit.ly/', 'http://fb.me/', 'http://ln.is/', 
             'http://goo.gl/', 'http://amzn.to/', 'http://ow.ly/', 'http://tinyurl.com/']
sorts_uri += [uri.replace('http', 'https') for uri in sorts_uri]
sorts_uri

['http://j.mp/',
 'http://bit.ly/',
 'http://fb.me/',
 'http://ln.is/',
 'http://goo.gl/',
 'http://amzn.to/',
 'http://ow.ly/',
 'http://tinyurl.com/',
 'https://j.mp/',
 'https://bit.ly/',
 'https://fb.me/',
 'https://ln.is/',
 'https://goo.gl/',
 'https://amzn.to/',
 'https://ow.ly/',
 'https://tinyurl.com/']

In [47]:
headers = {
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'User-Agent': ('Mozilla/5.0 (X11; Linux x86_64; rv:45.0)'
                   ' Gecko/20100101 Firefox/45.0'),
}

# so connections are recycled
session = requests.session()
session.headers = headers

def get_full_url(ele):
    if ele:
        parsed_uri = urlparse(ele)
        domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
        
        if domain in sorts_uri:
            resp = session.head(ele, allow_redirects=True)
            return resp.url
        else:
            return ele
    else:
        return False

df[df['external_url'] != False]['external_url'].apply(get_full_url)

6     https://twitter.com/i/web/status/8452676398406...
7     http://www.lanacion.com.ar/1403475-polemica-po...
15    https://twitter.com/docdrebin/status/844353272...
20            http://www.youtube.com/c/vpitvenvivo/live
24    https://twitter.com/i/web/status/8452726251108...
26    https://twitter.com/i/web/status/8452698343676...
35    https://www.cbo.gov/system/files/115th-congres...
38    https://twitter.com/i/web/status/8452694464994...
39    https://twitter.com/CFKArgentina/status/845272...
46    https://twitter.com/CFKArgentina/status/845272...
55    http://www.argnoticias.com/sociedad/La-musica-...
60    http://dcd67toxi0.ww3twb.s3.amazonaws.com/JLVF...
64    https://twitter.com/NEncuentroCABA/status/8452...
68    http://dcd67toxi0.ww3twb.s3.amazonaws.com/JLVF...
72    http://dcd67toxi0.ww3twb.s3.amazonaws.com/JLVF...
76            http://www.youtube.com/c/vpitvenvivo/live
78    https://www.baidu.com/link?url=pI4i43F-Aua7WuU...
81    https://twitter.com/i/web/status/845272674

In [35]:
df[df['external_url'] != False][:20]['external_url']

6     https://twitter.com/i/web/status/8452676398406...
7     http://www.lanacion.com.ar/1403475-polemica-po...
15    https://twitter.com/docdrebin/status/844353272...
20            http://www.youtube.com/c/vpitvenvivo/live
24    https://twitter.com/i/web/status/8452726251108...
26    https://twitter.com/i/web/status/8452698343676...
35    https://www.cbo.gov/system/files/115th-congres...
38    https://twitter.com/i/web/status/8452694464994...
39    https://twitter.com/CFKArgentina/status/845272...
46    https://twitter.com/CFKArgentina/status/845272...
55                                http://bit.ly/2nYMXAa
60                                  http://j.mp/2neCpg5
64    https://twitter.com/NEncuentroCABA/status/8452...
68                                  http://j.mp/2neCpg5
72                                  http://j.mp/2neCpg5
76            http://www.youtube.com/c/vpitvenvivo/live
78                           http://tinyurl.com/kml9otz
81    https://twitter.com/i/web/status/845272674

In [43]:
headers = {
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'User-Agent': ('Mozilla/5.0 (X11; Linux x86_64; rv:45.0)'
                   ' Gecko/20100101 Firefox/45.0'),
}

# so connections are recycled
session = requests.session()
session.headers = headers

resp = session.head('http://bit.ly/2nYMXAa', allow_redirects=True)
print(resp.url)

http://www.argnoticias.com/sociedad/La-musica-que-fue-censurada-y-prohibida-durante-la-dictadura-20170324-0006.html


In [41]:
resp.status_code

403